
## **Exercise 5.6: Multilevel Queueing with Different Time Quanta**

### **Question**
What advantage is there in having different time-quantum sizes at different levels of a multilevel queueing system?

### **Answer**
The primary advantage is **efficiency and performance optimization tailored to different process types**.

### **Detailed Explanation**
A **Multilevel Queue (MLQ)** algorithm partitions the ready queue into several separate queues, each with its own scheduling algorithm and priority. Assigning different time quanta to these queues allows the OS to match scheduling behavior to process characteristics:

1.  **High-Priority, Interactive Queues (e.g., foreground processes):** Given a **small time quantum** (e.g., 10-100 ms).
    *   **Why:** Interactive processes (editors, shells) require frequent, short CPU bursts to remain responsive to user input.
    *   **Benefit:** Small quanta ensure quick cycling among these processes, yielding good response times and a smooth user experience.

2.  **Low-Priority, Batch Queues (e.g., background processes):** Given a **large time quantum** or even FCFS.
    *   **Why:** CPU-bound batch jobs (scientific computations, compilations) benefit from long, uninterrupted runs to minimize overhead.
    *   **Benefit:** Large quanta drastically reduce the frequency of **context switches**, which are costly operations. This allows these processes to make more efficient use of the CPU and complete faster.

**Overall System Benefit:** This design provides a balanced system that delivers excellent interactive performance while also maximizing throughput for batch jobs, optimizing overall system efficiency.

---

## **Exercise 5.7: Relationships Between Algorithm Sets**

### **Question**
CPU-scheduling algorithms are often parameterized, forming sets of algorithms (e.g., all possible RR time quanta). One set may include another. What relation holds between the following pairs?

### **Answer & Explanation**

#### **a. Priority and SJF**
*   **Relation:** **SJF is a special case of Priority scheduling.**
*   **Explanation:** In Priority scheduling, the process with the highest priority number is selected. If we define a process's priority as the *inverse* of its predicted next CPU burst length (shorter burst = higher priority), then Priority scheduling becomes SJF.

#### **b. Multilevel Feedback Queues (MLFQ) and FCFS**
*   **Relation:** **FCFS is often used in at least one queue of an MLFQ system.**
*   **Explanation:** MLFQ typically uses RR with increasing time quanta in higher-level queues for interactive processes, and often employs **FCFS in the lowest-priority queue** for long-running CPU-bound background jobs. Thus, FCFS is contained within the set of possible MLFQ configurations.

#### **c. Priority and FCFS**
*   **Relation:** **FCFS can be seen as a special case of Priority scheduling.**
*   **Explanation:** If we define a process's priority as its *arrival time* (earlier arrival = higher priority), then the Priority scheduling algorithm will always select the process that arrived first, which is exactly FCFS.

#### **d. RR and SJF**
*   **Relation:** **No direct subset relationship.**
*   **Explanation:** RR is designed for fairness and preemption based on time, while SJF is designed for optimal average waiting time and can be non-preemptive or preemptive (SRTF). They are based on fundamentally different criteria (time slices vs. burst lengths). One is not a parameterized version of the other.

---


## **Exercise 5.9: PCS vs. SCS Scheduling**

### **Question**
Distinguish between PCS and SCS scheduling.

### **Answer**
*   **PCS (Process-Contention Scope):** Scheduling competition occurs **among threads belonging to the same process**. It is performed by the **thread library** at user-level.
*   **SCS (System-Contention Scope):** Scheduling competition occurs **among all threads in the system**. It is performed by the **operating system kernel**.

### **Detailed Explanation**
This distinction is crucial in multithreading models:

| Feature | PCS (User-Level) | SCS (Kernel-Level) |
| :--- | :--- | :--- |
| **Scheduler** | Thread library (e.g., pthreads). | OS kernel. |
| **Scope** | Local to the process. Threads compete for LWPs. | System-wide. Kernel threads compete for CPUs. |
| **Preemption** | Typically cooperative (non-preemptive) unless the library implements preemptive signals. | Preemptive, based on OS interrupts and time slices. |
| **Models** | Used in **Many-to-One** and **Many-to-Many** threading models. | Used in **One-to-One** model. In **Many-to-Many**, the kernel schedules LWPs (kernel threads) via SCS. |
| **Relation** | In Many-to-Many, PCS determines which user thread runs on an available LWP; SCS determines which LWP gets a CPU. | In the **One-to-One** model, PCS and SCS are effectively the same because each user thread maps to a kernel thread. |



## **Exercise 5.12: Conflicting Scheduling Criteria**

### **Question**
Discuss how the following pairs of scheduling criteria conflict in certain settings.

### **Answer & Explanation**

Scheduling algorithm design involves trade-offs. Optimizing for one metric often degrades another.

#### **a. CPU Utilization and Response Time**
*   **Conflict:** Maximizing CPU utilization can degrade response time, and vice-versa.
*   **Explanation:**
    *   **High CPU Utilization** is achieved by keeping the CPU busy at all times, often by favoring **CPU-bound processes** with long bursts or by minimizing context switches.
    *   **Good Response Time** (important for interactive systems) requires the CPU to be frequently available to handle short, incoming requests. This often means **preempting** long-running CPU-bound tasks to service interactive tasks, which introduces context switches and can leave the CPU idle briefly while switching.
    *   **Example:** In a **batch processing system** (goal: high utilization), using FCFS might keep the CPU 100% busy but lead to terrible response times for short interactive tasks. In a **time-sharing system** (goal: good response time), using RR with a small time quantum improves responsiveness but increases context switch overhead, potentially lowering overall CPU utilization.

#### **b. Average Turnaround Time and Maximum Waiting Time**
*   **Conflict:** Minimizing the average turnaround time can lead to an unacceptably high maximum waiting time for some processes (starvation).
*   **Explanation:**
    *   **Minimizing Average Turnaround Time** is the goal of algorithms like **SJF/SRTF**. They prioritize short jobs, which reduces the average.
    *   However, this can cause **indefinite postponement (starvation)** of long jobs. If short jobs keep arriving, a long job may wait indefinitely, causing its waiting time—and the system's **maximum waiting time**—to grow unbounded.
    *   **Example:** In a pure nonpreemptive SJF schedule, a single very long job will always be passed over if shorter jobs continue to arrive, maximizing its individual waiting time.




## **Exercise 5.14: Run Queue Design for Multicore Systems**

### **Question**
On multicore systems, there are two general options for the run queue: (1) per-core run queues, or (2) a single shared run queue. What are the advantages and disadvantages of each?

### **Answer**

| Approach | Advantages | Disadvantages |
| :--- | :--- | :--- |
| **Per-Core Run Queues** (Each core has its own private queue) | **1. Scalability:** No locking contention for the queue, allowing cores to schedule independently. <br> **2. Cache Locality:** A process tends to run on the same core, making better use of cached data. <br> **3. Reduced Overhead:** No need for complex locking mechanisms for queue access. | **1. Load Imbalance:** One core may be idle while another has a long queue. Requires a **load-balancing** mechanism. <br> **2. Complexity of Migration:** Moving a process from one core's queue to another's (for load balancing) is costly and can break cache locality. |
| **Single Shared Run Queue** (One global queue for all cores) | **1. Automatic Load Balancing:** Cores automatically take the next available process, naturally balancing load. <br> **2. Simplicity:** Conceptually simple, easy to implement fairness policies (e.g., global RR). | **1. Scalability Bottleneck:** The single queue becomes a **contended shared resource**. Cores must lock the queue to access it, which can serialize scheduling decisions and hurt performance as core count increases. <br> **2. Poor Cache Locality:** A process may execute on a different core each time it is scheduled, causing **cache misses**. |

**Conclusion:** Modern OSes typically use a **hybrid approach** (e.g., per-core queues with periodic load balancing) to mitigate the disadvantages of each pure design.




## **Exercise 5.17: Comprehensive Scheduling Comparison (Second Set)**

### **Question**
Consider the following processes (all arriving at time 0):

| Process | Burst Time | Priority |
| :--- | :---: | :---: |
| P1 | 5 | 4 |
| P2 | 3 | 1 |
| P3 | 1 | 2 |
| P4 | 7 | 2 |
| P5 | 4 | 3 |

*(Higher priority number = higher priority)*

**Tasks:**
a. Draw Gantt charts for: FCFS, SJF, Nonpreemptive Priority, RR (quantum=2).
b. Calculate turnaround time for each process under each algorithm.
c. Calculate waiting time for each process.
d. Identify which algorithm gives minimum average waiting time.

### **Answer**

#### **a. Gantt Charts**

1. **FCFS (First-Come, First-Served)**
   ```
   | P1 | P2 | P3 | P4  | P5   |
   0    5    8    9     16    20
   ```

2. **SJF (Shortest-Job-First) - Nonpreemptive**
   - Order by burst time: P3(1), P2(3), P5(4), P1(5), P4(7)
   ```
   | P3 | P2 | P5 | P1 | P4  |
   0    1    4    8   13    20
   ```

3. **Nonpreemptive Priority** (Higher number = higher priority)
   - Priorities: P1(4), P2(1), P3(2), P4(2), P5(3)
   - Order by priority: P1(4), P5(3), then P3 & P4 (both 2, tie-break by arrival → P3 then P4), then P2(1)
   ```
   | P1 | P5 | P3 | P4  | P2 |
   0    5    9   10    17   20
   ```

4. **RR (Round Robin, Quantum = 2)**
   - All processes arrive at time 0, order: P1, P2, P3, P4, P5
   ```
   |P1|P2|P3|P4|P5|P1|P2|P4|P5|P1|P4|P5|P4|
   0 2 4 5 6 8 10 11 13 14 16 18 19 20
   ```
   **Detailed RR Sequence:**
   - Time 0-2: P1 (3 left)
   - 2-4: P2 (1 left)
   - 4-5: P3 (1 burst, finishes at 5)
   - 5-6: P4 (5 left)
   - 6-8: P5 (2 left)
   - 8-10: P1 (1 left)
   - 10-11: P2 (finishes at 11)
   - 11-13: P4 (3 left)
   - 13-14: P5 (finishes at 14)
   - 14-16: P1 (finishes at 16)
   - 16-18: P4 (1 left)
   - 18-19: P4 (finishes at 19)
   - 19-20: [CPU idle? Actually P4 finished at 19, so idle from 19-20]

#### **b. Turnaround Time (TAT) Table**
*TAT = Completion Time - Arrival Time (Arrival=0 for all)*

| Process | FCFS | SJF | Priority | RR |
| :--- | :---: | :---: | :---: | :---: |
| P1 | 5 | 13 | 5 | 16 |
| P2 | 8 | 4 | 20 | 11 |
| P3 | 9 | 1 | 10 | 5 |
| P4 | 16 | 20 | 17 | 19 |
| P5 | 20 | 8 | 9 | 14 |

#### **c. Waiting Time (WT) Table**
*WT = Turnaround Time - Burst Time*

| Process | Burst | FCFS | SJF | Priority | RR |
| :--- | :---: | :---: | :---: | :---: | :---: |
| P1 | 5 | 0 | 8 | 0 | 11 |
| P2 | 3 | 5 | 1 | 17 | 8 |
| P3 | 1 | 8 | 0 | 9 | 4 |
| P4 | 7 | 9 | 13 | 10 | 12 |
| P5 | 4 | 16 | 4 | 5 | 10 |

#### **d. Algorithm with Minimum Average Waiting Time**
Calculate averages from part (c):

- **FCFS:** (0+5+8+9+16)/5 = 38/5 = **7.6**
- **SJF:** (8+1+0+13+4)/5 = 26/5 = **5.2**
- **Priority:** (0+17+9+10+5)/5 = 41/5 = **8.2**
- **RR:** (11+8+4+12+10)/5 = 45/5 = **9.0**

**SJF** gives the **minimum average waiting time (5.2)**, which is expected as SJF is optimal for minimizing average waiting time when all processes arrive simultaneously.

---




## **Exercise 5.26: Shared Ready Queue in SMP**

### **Question**
Describe why a shared ready queue might suffer from performance problems in an SMP environment.

### **Answer**
A shared ready queue becomes a **contention bottleneck** due to:
1. **Lock Synchronization:** Multiple processors must lock the queue to enqueue/dequeue processes, causing serialization and spin-waiting.
2. **Cache Coherence Overhead:** Frequent updates to the shared queue cause cache invalidation across processors, increasing memory latency.
3. **Scalability Limit:** As processor count grows, contention increases, reducing parallel efficiency.

**Solution:** Use per-processor run queues with work-stealing or load-balancing.

## **Exercise 5.28: Process Placement in Per-Processor Run Queues**

### **Question**
Assume that an SMP system has private, per-processor run queues. When a new process is created, it can be placed in either the same queue as the parent process or a separate queue.

a. What are the benefits of placing the new process in the same queue as its parent?
b. What are the benefits of placing the new process in a different queue?

### **Answer**

#### **a. Same Queue as Parent**
- **Cache Affinity:** Parent and child likely share code/data; keeping them on the same processor improves cache hit rates.
- **Synchronization Efficiency:** If they communicate frequently, shared memory accesses are faster within the same processor’s cache.

#### **b. Different Queue**
- **Load Distribution:** Spreads work across processors, improving overall throughput.
- **Parallel Execution:** Allows true concurrency if parent and child are independent.
- **Fault Isolation:** A busy processor doesn’t delay both processes.

**Trade-off:** Systems often use **affinity scheduling** (same queue initially) with **migration** for load balancing.

s

## **Exercise 5.37: Heterogeneous Multiprocessing in Mobile Systems**

### **Question**
Describe the advantages of using heterogeneous multiprocessing in a mobile system.

### **Answer**
**Heterogeneous Multiprocessing (HMP)** uses cores with different performance/power characteristics (e.g., ARM big.LITTLE: few "big" high-performance cores + many "LITTLE" power-efficient cores).

### **Advantages:**

1. **Energy Efficiency:** 
   - Light tasks → LITTLE cores (low power)
   - Heavy tasks → big cores (high performance)
   - Dynamic migration balances performance needs with battery life

2. **Thermal Management:**
   - Spread heat generation across different core types
   - Use LITTLE cores when thermal throttling required
   - Prevents overheating and performance degradation

3. **Performance Scaling:**
   - Fine-grained control over performance/power trade-off
   - Better than DVFS alone (dynamic voltage/frequency scaling)

4. **Cost Optimization:**
   - Mix of core types provides better performance/$ than homogeneous design
   - Smaller LITTLE cores occupy less die area

5. **Workload Matching:**
   - Background tasks (email sync, notifications) → LITTLE cores
   - Foreground tasks (gaming, camera) → big cores
   - OS scheduler intelligently places threads based on requirements

**Example:** Samsung Exynos/Qualcomm Snapdragon use HMP to achieve all-day battery life while providing burst performance when needed.
